In [39]:

import os
from glob import glob
import earthpy as et
import earthpy.appeears as eaapp
import geopandas as gpd
import hvplot.pandas
import hvplot.xarray
import requests
import rioxarray as rxr
import xarray as xr

In [3]:
grasslands_url =("https://data.fs.usda.gov/geodata/edw/"
                 "edw_resources/shp/S_USA.NationalGrassland.zip")
grasslands_gdf = gpd.read_file(grasslands_url)
grasslands_gdf

,NATIONALGR,GRASSLANDN,GIS_ACRES,SHAPE_AREA,SHAPE_LEN,geometry
0,295509010328,Sheyenne National Grassland,70428.175,0.033356,4.097398,"MULTIPOLYGON (((-97.31081 46.51457, -97.30559 ..."
1,295520010328,Little Missouri National Grassland,1025313.868,0.492746,60.420702,"MULTIPOLYGON (((-104.01688 47.51793, -104.0115..."
2,295513010328,Thunder Basin National Grassland,626249.208,0.282888,44.088050,"MULTIPOLYGON (((-105.46005 43.31908, -105.4601..."
3,295521010328,Oglala National Grassland,215804.927,0.096279,1.970612,"POLYGON ((-103.72477 43.00100, -103.72007 43.0..."
4,295519010328,Crooked River National Grassland,173593.026,0.079499,2.815873,"POLYGON ((-120.98025 44.72935, -120.98030 44.7..."
5,295515010328,Caddo National Grassland,68479.549,0.026940,1.159342,"MULTIPOLYGON (((-95.85492 33.79814, -95.85494 ..."
6,295523010328,Pawnee National Grassland,208424.885,0.089972,15.341594,"MULTIPOLYGON (((-104.58106 40.82664, -104.5810..."
7,295522010328,Comanche National Grassland,444413.904,0.183064,26.658022,"MULTIPOLYGON (((-104.02263 37.69224, -104.0225..."
8,295524010328,Cimarron National Grassland,109101.348,0.044765,5.539623,"MULTIPOLYGON (((-101.98977 37.11933, -101.9852..."
9,295508010328,Kiowa National Grassland,144281.321,0.058543,9.858642,"MULTIPOLYGON (((-104.30414 36.08063, -104.3041..."


In [5]:
min_lat,min_lon = 26, -99
max_lat,max_lon = min_lat+1, min_lon+1
polaris_url_format = ("http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/"
                      "v1.0/ph/mean/60_100/lat{min_lat}{max_lat}_lon{min_lon}{max_lon}.tif")
polaris_url = polaris_url_format.format(
    min_lat=min_lat, min_lon=min_lon, max_lat=max_lat, max_lon=max_lon)
polaris_url
rxr.open_rasterio(polaris_url, masked=True).squeeze()

<xarray.DataArray (y: 3600, x: 3600)>
[12960000 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 -99.0 -99.0 -99.0 -99.0 ... -98.0 -98.0 -98.0 -98.0
  * y            (y) float64 27.0 27.0 27.0 27.0 27.0 ... 26.0 26.0 26.0 26.0
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [18]:
cedar_river_gdf = (
    grasslands_gdf
    .set_index('GRASSLANDN').loc[['Cedar River National Grassland']]
)

download_key= 'Cedar-River-SRTM'
srtm_downloader = eaapp.AppeearsDownloader(
    download_key = download_key,
    product = 'SRTMGL1_NC.003',
    layer = 'SRTMGL1_DEM',
    start_date = '02-11-2000',
    end_date = '02-21-2000',
    polygon = cedar_river_gdf)
srtm_downloader.download_files()





In [22]:
srtm_paths = glob(
    os.path.join(
        srtm_downloader.data_dir,
        'SRTMGL1_NC.003*',
        '*.tif'))
[rxr.open_rasterio(srtm_path,masked=True).squeeze() for srtm_path in srtm_paths][0]

<xarray.DataArray (y: 470, x: 2283)>
[1073010 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 -101.9 -101.9 -101.9 ... -101.2 -101.2 -101.2
  * y            (y) float64 46.07 46.07 46.07 46.07 ... 45.95 45.94 45.94 45.94
    spatial_ref  int64 0
Attributes:
    add_offset:     0.0
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    units:          Meters

In [26]:
maca_url = ("http://thredds.northwestknowledge.net:8080/thredds/ncss/"
            "agg_macav2metdata_pr_CCSM4_r6i1p1_historical_1950_2005_"
            "CONUS_monthly.nc?var=precipitation&disableLLSubset=on&"
            "disableProjSubset=on&horizStride=1&time_start=1980-01-15T00"
            "%3A00%3A00Z&time_end=1980-12-15T00%3A00%3A00Z&timeStride="
            "1&accept=netcdf")
maca_response = requests.get(maca_url)
with open('maca.nc','wb') as maca_file:
    maca_file.write(maca_response.content)
xr.open_dataset('maca.nc')

<xarray.Dataset>
Dimensions:        (time: 12, lat: 585, lon: 1386)
Coordinates:
  * time           (time) object 1980-01-15 00:00:00 ... 1980-12-15 00:00:00
  * lat            (lat) float64 25.06 25.1 25.15 25.19 ... 49.31 49.35 49.4
  * lon            (lon) float64 235.2 235.3 235.3 235.4 ... 292.9 292.9 292.9
Data variables:
    precipitation  (time, lat, lon) float32 ...
Attributes: (12/48)
    description:                     Multivariate Adaptive Constructed Analog...
    id:                              MACAv2-METDATA
    naming_authority:                edu.uidaho.reacch
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    Metadata_Link:                   
    cdm_data_type:                   FLOAT
    ...                              ...
    publisher_email:                 reacch@uidaho.edu
    publisher_url:                   http://www.reacchpna.org/
    license:                         Creative Commons CC0 1.0 Universal Dedic...
    coordinate_system:               WGS84,EPSG:4326
    Conventions:                     CF-1.0
    History:                         Translated to CF-1.0 Conventions by Netc...

In [40]:
maca_ds = xr.open_dataset('maca.nc')

maca_ds =maca_ds = maca_ds.assign_coords(lon=maca_ds.lon-360)
precip_da = maca_ds.precipitation
precip_da.rio.write_crs("epsg:4326", inplace=True)
precip_da.rio.set_spatial_dims('lon','lat', inplace = True)

maca_ds.precipitation.mean('time').hvplot(rasterize=True)


BokehModel(combine_events=True, render_bundle={'docs_json': {'66a4d5f9-0cd7-4adb-8820-0affd806b55e': {'version…

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-60' coro=<Callback.process_on_change() done, defined at /opt/conda/lib/python3.10/site-packages/holoviews/plotting/bokeh/callbacks.py:355> exception=UnsetValueError("figure(id='p1345', ...).inner_height doesn't have a value set")>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/holoviews/plotting/bokeh/callbacks.py", line 374, in process_on_change
    msg[attr] = self.resolve_attr_spec(path, cb_obj)
  File "/opt/conda/lib/python3.10/site-packages/holoviews/plotting/bokeh/callbacks.py", line 281, in resolve_attr_spec
    resolved = getattr(resolved, p, None)
  File "/opt/conda/lib/python3.10/site-packages/bokeh/core/property/descriptors.py", line 283, in __get__
    raise UnsetValueError(f"{obj}.{self.name} doesn't have a value set")
bokeh.core.property.descriptors.UnsetValueError: figure(id='p1345', ...).inner_height doesn't have a value set

During handling of the abov

In [42]:
precip_da.rio.clip_box(*cedar_river_gdf.total_bounds).mean('time').hvplot()

:Image   [lon,lat]   (precipitation)